![rmotr](https://i.imgur.com/jiPp4hj.png)
<hr style="margin-bottom: 40px;">

<img src="https://user-images.githubusercontent.com/7065401/39117440-24199c72-46e7-11e8-8ffc-25c6e27e07d4.jpg"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Nettoyage de données avec Pandas

pandas emprunte toutes les capacités de numpy pour la sélection + ajoute un certain nombre de méthodes pratiques pour gérer les valeurs manquantes. Voyons-les une par une :


<img src= "https://i.imgur.com/4gX5WFr.png"/>

## Let's go !

In [7]:
import numpy as np
import pandas as pd

### Fonctions utilitaires de Pandas

Tout comme `numpy`, pandas a également quelques fonctions utilitaires pour identifier et détecter les valeurs nulles :

In [9]:
pd.isnull(np.nan)

In [10]:
pd.isnull(None)

In [11]:
pd.isna(np.nan)

In [12]:
pd.isna(None)

Les fonctions opposées existent également :

In [13]:
pd.notnull(None)

In [14]:
pd.notnull(np.nan)

In [15]:
pd.notna(np.nan)

In [16]:
pd.notnull(3)

Ces fonctions fonctionnent également avec les Series et les `DataFrame` :

In [17]:
pd.isnull(pd.Series([1, np.nan, 7]))

In [18]:
pd.notnull(pd.Series([1, np.nan, 7]))

In [19]:
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

![separator1](https://i.imgur.com/ZUWYTii.png)

### Opérations Pandas avec valeurs manquantes

Pandas gère les valeurs manquantes plus élégamment que numpy. Les `nan` ne se comporteront plus comme des "virus", et les opérations les ignoreront simplement complètement :

In [ ]:
pd.Series([1, 2, np.nan]).count()

In [ ]:
pd.Series([1, 2, np.nan]).sum()

In [ ]:
pd.Series([2, 2, np.nan]).mean()

### Filtrer les données manquantes

Comme nous l'avons vu avec numpy, nous pouvons combiner la sélection booléenne + `pd.isnull` pour filtrer ces `nan` et valeurs nulles :

In [13]:
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])

In [20]:
pd.notnull(s)

In [23]:
pd.isnull(s)

In [24]:
pd.notnull(s).sum()

In [25]:
pd.isnull(s).sum()

In [28]:
s[pd.notnull(s)]

Mais `notnull` et `isnull` sont aussi des méthodes de `Series` et `DataFrame`, nous pouvons donc les utiliser de cette manière :

In [29]:
s.isnull()

In [30]:
s.notnull()

In [31]:
s[s.notnull()]

![separator1](https://i.imgur.com/ZUWYTii.png)

### Supprimer les valeurs nulles

La sélection booléenne + `notnull()` semble un peu verbeuse et répétitive. Et comme nous l'avons dit précédemment : toute tâche répétitive aura probablement une meilleure façon, plus DRY. Dans ce cas, nous pouvons utiliser la méthode `dropna` :

In [32]:
s

In [33]:
s.dropna()

### Supprimer les valeurs nulles dans les DataFrames

Vous avez vu à quel point il est simple de supprimer les `na` avec une Series. Mais avec les `DataFrame`, il y aura quelques éléments supplémentaires à considérer, car vous ne pouvez pas supprimer des valeurs individuelles. Vous ne pouvez supprimer que des colonnes ou des lignes entières. Commençons par un exemple de `DataFrame` :

In [34]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [35]:
df

In [36]:
df.shape

In [37]:
df.info()

In [38]:
df.isnull()

In [39]:
df.isnull().sum()

Le comportement par défaut de `dropna` supprimera toutes les lignes dans lesquelles _n'importe quelle_ valeur nulle est présente :

In [40]:
df.dropna()

Dans ce cas, nous supprimons des **lignes**. Les lignes contenant des valeurs nulles sont supprimées du DF. Vous pouvez également utiliser le paramètre `axis` pour supprimer les colonnes contenant des valeurs nulles :

In [43]:
df.dropna(axis=1)  # axis='columns' also works

Dans ce cas, toute ligne ou colonne qui contient **au moins** une valeur nulle sera supprimée. Ce qui peut être, selon le cas, trop extrême. Vous pouvez contrôler ce comportement avec le paramètre `how`. Peut être soit `'any'` soit `'all'` :

In [ ]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

In [44]:
df2

In [46]:
df.dropna(how='all')

In [47]:
df.dropna(how='any')  # défaut

Vous pouvez également utiliser le paramètre `thresh` pour indiquer un _seuil_ (un nombre minimum) de valeurs non nulles pour que la ligne/colonne soit conservée :

In [ ]:
df

In [48]:
df.dropna(thresh=3)

In [49]:
df.dropna(thresh=3, axis='columns')

![separator1](https://i.imgur.com/ZUWYTii.png)

### Remplir les valeurs nulles

Parfois, au lieu de supprimer les valeurs nulles, nous pourrions avoir besoin de les remplacer par une autre valeur. Cela dépend fortement de votre contexte et du jeu de données sur lequel vous travaillez actuellement. Parfois un `nan` peut être remplacé par un `0`, parfois il peut être remplacé par la `moyenne` de l'échantillon, et d'autres fois vous pouvez prendre la valeur la plus proche. Encore une fois, cela dépend du contexte. Nous vous montrerons les différentes méthodes et mécanismes et vous pourrez ensuite les appliquer à votre propre problème.

In [50]:
s

**Remplir les valeurs nulles avec une valeur arbitraire**

In [51]:
s.fillna(0)

In [52]:
s.fillna(s.mean())

In [ ]:
s

**Remplir les valeurs nulles avec des valeurs contiguës (proches)**

L'argument `method` est utilisé pour remplir les valeurs nulles avec d'autres valeurs proches de cette valeur nulle :

In [53]:
s.fillna(method='ffill')

In [54]:
s.fillna(method='bfill')

Cela peut encore laisser des valeurs nulles aux extrémités de la Series/DataFrame :

In [55]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

In [56]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')

### Remplir les valeurs nulles dans les DataFrames

La méthode `fillna` fonctionne également sur les `DataFrame`, et elle fonctionne de manière similaire. Les principales différences sont que vous pouvez spécifier l'`axis` (comme d'habitude, lignes ou colonnes) à utiliser pour remplir les valeurs (spécialement pour les méthodes) et que vous avez plus de contrôle sur les valeurs transmises :

In [ ]:
df

In [57]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

In [58]:
df.fillna(method='ffill', axis=0)

In [59]:
df.fillna(method='ffill', axis=1)

![separator1](https://i.imgur.com/ZUWYTii.png)

### Vérifier s'il y a des NAs

La question est : Cette `Series` ou ce `DataFrame` contient-il des valeurs manquantes ? La réponse devrait être oui ou non : `True` ou `False`. Comment pouvez-vous le vérifier ?

**Exemple 1 : Vérifier la longueur**

S'il y a des valeurs manquantes, `s.dropna()` aura moins d'éléments que `s` :

In [62]:
s.dropna().count()

In [ ]:
missing_values = len(s.dropna()) != len(s)
missing_values

Il y a aussi une méthode `count`, qui exclut les `nan` de son résultat :

In [ ]:
len(s)

In [ ]:
s.count()

Nous pourrions donc simplement faire :

In [ ]:
missing_values = s.count() != len(s)
missing_values

**Solution plus Pythonique avec `any`**

Les méthodes `any` et `all` vérifient s'il y a `any` (n'importe quelle) valeur True dans une Series ou si `all` (toutes) les valeurs sont `True`. Elles fonctionnent de la même manière qu'en Python :

In [63]:
pd.Series([True, False, False]).any()

In [64]:
pd.Series([True, False, False]).all()

In [65]:
pd.Series([True, True, True]).all()

La méthode `isnull()` a retourné une `Series` booléenne avec des valeurs `True` partout où il y avait un `nan` :

In [ ]:
s.isnull()

Nous pouvons donc simplement utiliser la méthode `any` avec le tableau booléen retourné :

In [ ]:
pd.Series([1, np.nan]).isnull().any()

In [ ]:
pd.Series([1, 2]).isnull().any()

In [ ]:
s.isnull().any()

Une version plus stricte vérifierait uniquement les `values` de la Series :

In [ ]:
s.isnull().values

In [ ]:
s.isnull().values.any()

![separator2](https://i.imgur.com/4gX5WFr.png)